In [30]:
import unittest
import farabio
from farabio.data.biodatasets import ChestXrayDataset, DSB18Dataset, HistocancerDataset, RANZCRDataset, RetinopathyDataset

In [6]:
_path = "/home/data/02_SSD4TB/suzy/datasets/public/chest-xray"
valid_dataset = ChestXrayDataset(root=_path, download=False, mode="val", show=False)

In [20]:
_path = "/home/data/02_SSD4TB/suzy/datasets/public/data-science-bowl-2018"
train_dataset = DSB18Dataset(root=_path, transform=None, mode="train", download=False, show=False)

In [24]:
_path = "/home/data/02_SSD4TB/suzy/datasets/public/histopathologic-cancer-detection"
train_dataset = HistocancerDataset(root=_path, download=False, mode="train", show=False)

In [28]:
_path = "/home/data/02_SSD4TB/suzy/datasets/public/ranzcr-clip-catheter-line-classification"
train_dataset = RANZCRDataset(root=_path, show=False, shape=512, mode="train")

In [32]:
_path = "/home/data/02_SSD4TB/suzy/datasets/public/aptos2019-blindness-detection"
train_dataset = RetinopathyDataset(root=_path, mode="train", show=False)

In [29]:
len(train_dataset)

14399

In [11]:


class TestBiodatasets(unittest.TestCase):
    def testChestXrayDataset(self):
        _path = "/home/data/02_SSD4TB/suzy/datasets/public/chest-xray"
        valid_dataset = ChestXrayDataset(root=_path, download=False, mode="val", show=False)
        print(valid_dataset)
    
    def testDSB18Dataset(self):
        _path = "/home/data/02_SSD4TB/suzy/datasets/public/data-science-bowl-2018"
        train_dataset = DSB18Dataset(root=_path, transform=None, mode="train", download=False, show=False)
        print(train_dataset)
    
    def testHistocancerDataset(self):
        _path = "/home/data/02_SSD4TB/suzy/datasets/public/histopathologic-cancer-detection"
        train_dataset = HistocancerDataset(root=_path, download=False, mode="train", show=False)
        print(train_dataset)
        
    def testRANZCRDataset(self):
        _path = "/home/data/02_SSD4TB/suzy/datasets/public/ranzcr-clip-catheter-line-classification"
        train_dataset = RANZCRDataset(root=_path, show=False, shape=512, mode="train")
        print(train_dataset)
    
    def testRetinopathyDataset(self):
        _path = "/home/data/02_SSD4TB/suzy/datasets/public/aptos2019-blindness-detection"
        train_dataset = RetinopathyDataset(root=_path, mode="train", show=False)
        print(train_dataset)

if __name__ == '__main__':
    unittest.main()

E
ERROR: /home/suzy/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/suzy/'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/home/suzy/miniconda3/envs/fclean/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
